<a href="https://colab.research.google.com/github/michkowalczuk/dw_matrix_road_signs/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#####Imports

In [1]:
!pip install hyperopt

In [0]:
import os
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from skimage import color, exposure

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPool2D, Flatten,Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

#####Import data

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_3/dw_matrix_road_signs'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_3/dw_matrix_road_signs


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

[X_train, y_train] = train['features'], train['labels']
[X_test, y_test] = test['features'], test['labels']

In [0]:
df=pd.read_csv('data/signnames.csv')
labels = df.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

#####Define functions

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),  # rozmiar paczki
      epochs=params_fit.get('epochs', 5),            # liczba iteracji
      verbose=params_fit.get('verbose', 1),          # stan
      validation_data=params_fit.get('validation_data', (X_train, y_train)),  # do krzywych uczenia się
      callbacks=[tensorboard_callback]               # do wizualizacji
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax')
  ])

In [10]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)  

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.6077 - accuracy: 0.2784 - val_loss: 1.0941 - val_accuracy: 0.6525
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.8412 - accuracy: 0.7295 - val_loss: 0.2136 - val_accuracy: 0.9342
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3969 - accuracy: 0.8766 - val_loss: 0.0973 - val_accuracy: 0.9733
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2477 - accuracy: 0.9261 - val_loss: 0.0641 - val_accuracy: 0.9831
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.1885 - accuracy: 0.9438 - val_loss: 0.0307 - val_accuracy: 0.9913


0.9643990929705215

In [11]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.1745 - accuracy: 0.9644


[0.1745445430278778, 0.9643990993499756]

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_1']),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_2']),
    
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_3']),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_1']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_2']),
    
    Dense(num_classes, activation='softmax')
  ])

#####hyperopt

In [0]:
def func_obj(params):
  get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),  # rozmiar paczki
      epochs=5,            # liczba iteracji
      verbose=0,          # stan
  )

  score = model.evaluate(X_test, y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [16]:
space = {
      'batch_size': hp.quniform('batch_size', 100, 200, 10),
      'dropout_cnn_block_1': hp.uniform('dropout_cnn_block_1', 0.3, 0.5),
      'dropout_cnn_block_2': hp.uniform('dropout_cnn_block_2', 0.3, 0.5),
      'dropout_cnn_block_3': hp.uniform('dropout_cnn_block_3', 0.3, 0.5),
      'dropout_dense_block_1': hp.uniform('dropout_dense_block_1', 0.3, 0.7),
      'dropout_dense_block_2': hp.uniform('dropout_dense_block_2', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 150.0, 'dropout_cnn_block_1': 0.4927495754082575, 'dropout_cnn_block_2': 0.45027851838754257, 'dropout_cnn_block_3': 0.3165836266923972, 'dropout_dense_block_1': 0.5381581427755003, 'dropout_dense_block_2': 0.38474956655432146}
accuracy=0.974829912185669
{'batch_size': 130.0, 'dropout_cnn_block_1': 0.3302322487998526, 'dropout_cnn_block_2': 0.41812692043789323, 'dropout_cnn_block_3': 0.3821838933670423, 'dropout_dense_block_1': 0.6463347575701813, 'dropout_dense_block_2': 0.49140470404436415}
accuracy=0.976643979549408
{'batch_size': 180.0, 'dropout_cnn_block_1': 0.38822350312899445, 'dropout_cnn_block_2': 0.45046959022093414, 'dropout_cnn_block_3': 0.4891409781051562, 'dropout_dense_block_1': 0.6658019422900859, 'dropout_dense_block_2': 0.5027208541348206}
accuracy=0.9832199811935425
{'batch_size': 140.0, 'dropout_cnn_block_1': 0.330571019596842, 'dropout_cnn_block_2': 0.32308005924588407, 'dropout_cnn_block_3': 0.30851862574091565, 'dropout_dense_block_1': 0.6292041039